# Scikit-Learn IRIS Model using jsonData

**Goal**: Wrap a scikit-learn python model for use as a prediction microservice in seldon-core

#### Steps

1. Run locally on Docker to test
2. Deploy on seldon-core running on a kubernetes cluster

#### Setup Python

In [56]:
%%writefile requirements.txt
scikit-learn
spacy
dill
pandas
sklearn
seldon-core
pycurl

Overwriting requirements.txt


In [62]:
!if [ ! -d "./venv" ]; then python -m venv venv; fi

In [63]:
!source ./venv/bin/activate

In [65]:
!if [ ! -d "/home/jovyan/.config/pip" ]; then mkdir /home/jovyan/.config/pip; fi

In [52]:
%%writefile /home/jovyan/.config/pip/pip.conf
[global]
index-url = https://jfrog.aaw.cloud.statcan.ca/artifactory/api/pypi/pypi-remote/simple

Overwriting /home/jovyan/.config/pip/pip.conf


In [35]:
%%capture
!pip install -r requirements.txt

In [15]:
%%capture
!python -m spacy download en_core_web_sm

In [16]:
!python -m ipykernel install --user --name=sklearn_iris_jsondata

Installed kernelspec sklearn_iris_jsondata in /etc/share/jupyter/kernels/sklearn_iris_jsondata


## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

```
kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80
```

Create a seldon config file to deploy the containerized image you just created

In [72]:
%%writefile sklearn_iris_jsondata_deployment.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: seldon-deployment-example
  namespace: bryanpaget
spec:
  name: sklearn-iris-deployment
  annotations:
    sidecar.istio.io/inject: "false"
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/sklearn-iris:0.3
          imagePullPolicy: IfNotPresent
          name: sklearn-iris-classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: sklearn-iris-classifier
      type: MODEL
    name: sklearn-iris-predictor
    replicas: 1

Overwriting sklearn_iris_jsondata_deployment.yaml


In [73]:
!kubectl create -f sklearn_iris_jsondata_deployment.yaml

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [74]:
!kubectl rollout status deploy $(kubectl get deploy -l seldon-deployment-id=seldon-deployment-example -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" rollout to finish: 0 out of 1 new replicas have been updated...
Waiting for deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-92a927e5e90d7602e08ba9b9304f70e8" successfully rolled out


### Test by sending prediction calls

jsonData sent as json

In [86]:
for i in range(60):
    state = !kubectl get sdep seldon-deployment-example -o jsonpath='{.status.state}'
    state = state[0]
    print(state)
    if state == "Available":
        break
    time.sleep(1)
assert state == "Available"

Available


In [87]:
res = !curl -s http://localhost:8003/seldon/seldon/seldon-deployment-example/api/v0.1/predictions -H "Content-Type: application/json" -d '{"some_data": {"names": ["sepal_length","sepal_width","petal_length","petal_width"],"some_ndarray": [[7.233,4.652,7.39,0.324]]}}'

In [88]:
print(res)

[]


jsonData sent as form-data

In [71]:
!kubectl delete -f sklearn_iris_jsondata_deployment.yaml

seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted
